In [6]:
# default_exp exec.overlay_audios

In [41]:
# export
import argparse
import sys
import librosa  # NOTE(zach): importing torch before librosa causes LLVM issues for some unknown reason.
import json
import os
import glob

from pydub import AudioSegment

def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("-d","--data-folder", help="Path to folder with .wav files")
    parser.add_argument("-b","--background" help="Path to single .wav background file")
    args = parser.parse_args(args)
    return args

In [60]:
# export
def run(hparams):

    music_audio_segment = AudioSegment.from_file(args.background)

    for file in os.listdir(args.data_folder):
        if os.path.splitext(file)[-1].lower() == ".wav":

            played_togther = vocal_audio_segment.overlay(music_audio_segment)

    # Create model
    model = GradTTS(hparams)
    model.load_state_dict(torch.load(hparams.checkpoint))
    model = model.cuda()

    stft = MelSTFT(
        filter_length=hparams.filter_length,
        hop_length=hparams.hop_length,
        win_length=hparams.win_length,
        n_mel_channels=hparams.n_feats,
        sampling_rate=hparams.sampling_rate,
        mel_fmin=hparams.mel_fmin,
        mel_fmax=hparams.mel_fmax,
        padding=(hparams.filter_length - hparams.hop_length) // 2,
    )

    hifigan = HiFiGanGenerator(
        config=hparams.hifigan_config,
        checkpoint=hparams.hifigan_checkpoint,
        cudnn_enabled=True,
    )

    # Get reference audio mel spectrogram
    #     sample_rate, wav_data = read(hparams.reference_audio)
    vocal_data, vocal_sample_rate = librosa.load(
        hparams.reference_vocals, sr=hparams.sampling_rate
    )
    music_data, music_sample_rate = librosa.load(
        hparams.reference_music, sr=hparams.sampling_rate
    )
    music_data = music_data / hparams.max_wav_value
    assert vocal_sample_rate == music_sample_rate

    #     audio_norm = torch.FloatTensor(wav_data) / hparams.max_wav_value
    audio_norm = torch.FloatTensor(vocal_data) / hparams.max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    melspec_original = stft.mel_spectrogram(audio_norm).cuda()

    with open(hparams.customizations, "r") as csvfile:
        datareader = csv.reader(csvfile)
        for i, row in enumerate(datareader):
            regex = r"\|(.*?)\|"
            substitution = f"| {row[1]} |"
            new_transcription = re.sub(regex, substitution, hparams.transcription)
            print(new_transcription)

            y_dec1, y_dec2, y_dec_edit, y_dec_cat = model.infer_editts_edit_content(
                hparams.transcription,
                new_transcription,
                n_timesteps=10,
                symbol_set="gradtts",
                mel1=melspec_original.cuda(),
            )
            personalized_vocals = hifigan.infer(y_dec_edit)

            #             vocal_pydub = AudioSegment.from
            vocal_audio_segment = AudioSegment(
                personalized_vocals.tobytes(),
                frame_rate=vocal_sample_rate,
                sample_width=personalized_vocals.dtype.itemsize,
                channels=1,
            )

            music_audio_segment = AudioSegment.from_file(hparams.reference_music)

            played_togther = vocal_audio_segment.overlay(music_audio_segment)

            #             final_audio = overlay(personalized_vocals, music_data)
            #             sound.get_array_of_samples()
            #             melspec = stft.mel_spectrogram(audio_norm).cuda()
            #             plot_spectrogram(melspec.squeeze().cpu())
            #             plt.show()
            played_together.export(f"{hparams.log_dir}/edited_{i}.wav")


#             write(f"{hparams.log_dir}/edited_{i}.wav", hparams.sampling_rate, final_audio)

In [61]:
# export
try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    config = GRADTTS_DEFAULTS.values()
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
    hparams = HParams(**config)
    run(hparams)

In [62]:
config = GRADTTS_DEFAULTS.values()
with open("../configs/editts.json") as f:
    config.update(json.load(f))
hparams = HParams(**config)
run(hparams)

This year you won | one hundred | now that's a stack


NameError: name 'played_together' is not defined

In [ ]:
# def overlay(audio1, audio2):
#     audio1_padded = np.pad(audio1, (0,max(0,len(audio2)-len(audio1))))
#     audio2_padded = np.pad(audio2, (0,max(0,len(audio1)-len(audio2))))
#     return audio1_padded + audio2_padded

In [ ]:
from pydub import AudioSegment
import numpy as np

wav1, s1 = librosa.load("../data/beat_trim.wav", sr=22050)
wav2, s2 = librosa.load("../data/vocal_trim.wav", sr=22050)

assert s1 == s2, "sample rates must be equal"
overlay_wav = overlay(wav1, wav2)

In [ ]:
import IPython.display as ipd

ipd.Audio(overlay_wav, rate=22050)  # load a local WAV file

In [11]:
ipd.Audio(wav1, rate=22050)  # load a local WAV file
#

NameError: name 'ipd' is not defined

In [12]:
ipd.Audio(wav2, rate=22050)  # load a local WAV file

NameError: name 'ipd' is not defined

In [ ]:
x = np.array([1, 1, 1])
y = np.array([2, 2, 2, 2, 2, 2])

x = np.pad(x, (0, len(y) - len(x)))
y = np.pad(y, (0, len(x) - len(y)))

print(x, y)
print(z.shape)

print(z + y)